In [1]:
import torch
import sys
sys.path.append('rat-sql')
sys.path.append('../')

import attr
import _jsonnet
import json 

from ratsql.commands.infer import Inferer
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "3"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"

os.chdir('/repo')

2022-09-16 00:34:30.085613: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
@attr.s
class InferConfig:
    config = attr.ib()
    config_args = attr.ib()
    logdir = attr.ib()
    section = attr.ib()
    beam_size = attr.ib()
    output = attr.ib()
    step = attr.ib()
    use_heuristic = attr.ib(default=False)
    mode = attr.ib(default="infer")
    limit = attr.ib(default=None)
    output_history = attr.ib(default=False)

infer_config = InferConfig(config='rat-sql/configs/spider/nl2code-bert.jsonnet', 
                            config_args='{"att": 1, "bert_lr": 3e-06, "bert_token_type": true, "bert_version": "bert-large-uncased-whole-word-masking", "bs": 12, "clause_order": null, "cv_link": true, "data_path": "rat-sql/data/spider/", "decoder_hidden_size": 512, "end_lr": 0, "end_with_from": true, "loss": "label_smooth", "lr": 0.000744, "max_steps": 121000, "num_batch_accumulated": 4, "num_layers": 8, "qv_link": false, "dist_relation": true, "sc_link": true, "summarize_header": "avg", "use_align_loss": true, "use_align_mat": true, "use_column_type": false}', 
                            logdir='rat-sql/logdir/bert_run', 
                            section='val',
                            beam_size=1, 
                            output='__LOGDIR__/ie_dirs/bert_run_true_1-step1.infer', 
                            step=1, use_heuristic=True, mode='infer', limit=None, output_history=False)


In [3]:
config = json.loads(_jsonnet.evaluate_file(infer_config.config, tla_codes={'args': infer_config.config_args}))

inferer = Inferer(config)

WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


In [4]:
logdir = os.path.join(infer_config.logdir, config['model_name'])
step = 10100

model = inferer.load_model(logdir, step)

WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bert_version': 'bert-large-uncased-whole-word-masking', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'rat-sql/data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink'}}


Used Relations: 51


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from rat-sql/logdir/bert_run/bs=12,lr=7.4e-04,bert_lr=3.0e-06,loss=label_smooth,qv_link=false,dist=true/model_checkpoint-00010100


In [5]:
relation_k_emb_7 = model.state_dict()['encoder.encs_update.encoder.layers.7.relation_k_emb.weight']

In [6]:
from torch.utils.tensorboard import SummaryWriter


# 기본 `log_dir` 은 "runs"이며, 여기서는 더 구체적으로 지정하였습니다
writer = SummaryWriter('rat-sql/tf_runs/ratsql_embedding_vis')

In [7]:
relation_index = {('qq_dist', -2): 0,
('qq_dist', -1): 1,
('qq_dist', 0): 2,
('qq_dist', 1): 3,
('qq_dist', 2): 4,
'qc_default': 5,
'qt_default': 6,
'cq_default': 7,
'cc_default': 8,
'cc_foreign_key_forward': 9,
'cc_foreign_key_backward': 10,
'cc_table_match': 11,
('cc_dist', -2): 12,
('cc_dist', -1): 13,
('cc_dist', 0): 14,
('cc_dist', 1): 15,
('cc_dist', 2): 16,
'ct_default': 17,
'ct_foreign_key': 18,
'ct_primary_key': 19,
'ct_table_match': 20,
'ct_any_table': 21,
'tq_default': 22,
'tc_default': 23,
'tc_primary_key': 24,
'tc_table_match': 25,
'tc_any_table': 26,
'tc_foreign_key': 27,
'tt_default': 28,
'tt_foreign_key_forward': 29,
'tt_foreign_key_backward': 30,
'tt_foreign_key_both': 31,
('tt_dist', -2): 32,
('tt_dist', -1): 33,
('tt_dist', 0): 34,
('tt_dist', 1): 35,
('tt_dist', 2): 36,
'qcCEM': 37,
'cqCEM': 38,
'qtTEM': 39,
'tqTEM': 40,
'qcCPM': 41,
'cqCPM': 42,
'qtTPM': 43,
'tqTPM': 44,
'qcNUMBER': 45,
'cqNUMBER': 46,
'qcTIME': 47,
'cqTIME': 48,
'qcCELLMATCH': 49,
'cqCELLMATCH': 50}

In [8]:
relation_index_list = relation_index.keys()

In [9]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

writer.add_embedding(relation_k_emb_7,
                    metadata=relation_index_list)

In [10]:
import os
import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorboard.plugins import projector

# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='rat-sql/tf_runs/relation_emb_vis/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for key in relation_index_list:
    f.write("{}\n".format(key))
  # Fill in the rest of the labels with "unknown".


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(relation_k_emb_7.cpu())
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

2022-09-16 00:34:56.977884: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-16 00:34:56.978029: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-16 00:34:56.990682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-16 00:34:56.990770: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [11]:
!tensorboard --logdir=rat-sql/tf_runs/relation_emb_vis --port 6066 --host=0.0.0.0

2022-09-16 00:34:58.744357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-16 00:34:58.744402: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-16 00:35:00.649861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-16 00:35:00.649965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local